In [ ]:
import scipy.io
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import tensorflow as tf
from models.VGG_16 import VGG_16
import keras
from keras.preprocessing import image
from keras.callbacks import ModelCheckpoint,EarlyStopping
from keras.layers import Dense, Activation, Dropout, Flatten, Input, Convolution2D, ZeroPadding2D, MaxPooling2D, Activation
from keras.layers import Conv2D, AveragePooling2D, Concatenate, Lambda
from keras.models import Model, Sequential
from sklearn.model_selection import train_test_split
from keras.preprocessing.image import ImageDataGenerator
from script.mixup_generator import MixupGenerator
from script.random_eraser import get_random_eraser
from keras import metrics
from collections import defaultdict
from keras.models import model_from_json
import matplotlib.pyplot as plt
import torch
import cv2
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
keras.backend.clear_session()

In [ ]:
classes = 101 #(0, 100])
print("number of output classes: ",classes)

In [ ]:
features=torch.load('./data/features.pt').numpy()
target=torch.load('./data/age.pt').numpy()

In [ ]:
target_classes = keras.utils.to_categorical(target, classes)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(features, target_classes, test_size=0.30)

In [ ]:
#delete extra data
del features
del target
del target_classes

## Training for Age

In [ ]:
# Set data augmentation
aug=False
# set baseline
baseline=True

# Normalize data
train_x/=255
test_x/=255

# Set epochs and batch_size
epochs = 30
batch_size = 256
size_train=train_x.shape[0]

age_model=VGG_16(101)(pretrained=True)

if aug:
    batch_size=32
    datagen = ImageDataGenerator(
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,
            preprocessing_function=get_random_eraser(v_l=0, v_h=255))
    
    training_generator = MixupGenerator(train_x, train_y, batch_size=batch_size, alpha=0.2,
                                            datagen=datagen)()

In [ ]:
sgd = keras.optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

age_model.compile(loss='categorical_crossentropy'
                  , optimizer=sgd
                  , metrics=['accuracy']
                 )

In [ ]:
enableFit = True

if enableFit:
    if aug:
        score = age_model.fit_generator(generator=training_generator,
                                   steps_per_epoch=size_train // batch_size,
                                   validation_data=(test_x, test_y),
                                   epochs=epochs, verbose=1)
        pd.DataFrame.from_dict(score.history).to_csv('./outputs/baseline_age_aug.csv')

        plt.plot(score.history['val_loss'], label='val_loss')
        plt.plot(score.history['loss'], label='train_loss')
        plt.legend(loc='upper right')
        plt.ylim([2.5,5])
        plt.savefig('baseline_age_aug.png')
        plt.show()

        
    elif baseline:

        scores=defaultdict(list)
        score = age_model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, validation_data=(test_x, test_y))
        model='_'
        age_model.save_weights('./trained_models/baseline_age_'+model+'.h5')
        pd.DataFrame.from_dict(scores).to_csv('./outputs/baseline_age_'+model+'.csv')

        plt.plot(score.history['loss'], label='loss')
        plt.plot(score.history['val_loss'], label='val_loss')
        plt.legend(loc='upper right')   
        plt.savefig('./outputs/baseline_age_'+model+'.png')
        plt.show()

else:
    #pre-trained weights for age prediction: https://drive.google.com/file/d/1YCox_4kJ-BYeXq27uUbasu--yz28zUMV/view?usp=sharing
    age_model.load_weights("./trained_models/age_model_weights.h5")

## Model Prediction for Age

In [ ]:
predictions = age_model.predict(test_x)

In [ ]:
output_indexes = np.array([i for i in range(0, 101)])
apparent_predictions = np.sum(predictions * output_indexes, axis = 1)

In [ ]:
mae = 0
actual_mean = 0

for i in range(0 ,apparent_predictions.shape[0]):
    prediction = int(apparent_predictions[i])
    actual = np.argmax(test_y[i])
    abs_error = abs(prediction - actual)
    actual_mean = actual_mean + actual
    mae = mae + abs_error
    
mae = mae / apparent_predictions.shape[0]

print("mae: ",mae)
print("instances: ",apparent_predictions.shape[0])

## Training for Gender

In [ ]:
features=torch.load('./data/features.pt').numpy()
gender=torch.load('./data/gender.pt').numpy()
gender_classes = keras.utils.to_categorical(gender, 2)

In [ ]:
train_x, test_x, train_y, test_y = train_test_split(features, gender_classes, test_size=0.30)

In [ ]:
del features
del gender_classes

In [ ]:
aug=False
baseline=True

train_x/=255
test_x/=255

epochs = 30
batch_size = 256
size_train=train_x.shape[0]

gender_model=VGG_16(2)(pretrained=True)

if aug:
    batch_size=32
    datagen = ImageDataGenerator(
            width_shift_range=0.1,
            height_shift_range=0.1,
            horizontal_flip=True,
            preprocessing_function=get_random_eraser(v_l=0, v_h=255))

    training_generator = MixupGenerator(train_x, train_y, batch_size=batch_size, alpha=0.2,
                                            datagen=datagen)()

In [ ]:
sgd = keras.optimizers.SGD(lr=1e-3, decay=1e-6, momentum=0.9, nesterov=True)

gender_model.compile(loss='categorical_crossentropy'
                  , optimizer=keras.optimizers.Adam()
                  #, optimizer = sgd
                  , metrics=['accuracy']
                 )

In [ ]:
enableFit = True

if enableFit:
    if aug:
        score = gender_model.fit_generator(generator=training_generator,
                               steps_per_epoch=size_train // batch_size,
                               validation_data=(test_x, test_y),
                               epochs=epochs, verbose=1)

        pd.DataFrame.from_dict(score.history).to_csv('./outputs/baseline_gender_aug.csv')
        plt.plot(score.history['val_loss'], label='val_loss')
        plt.plot(score.history['loss'], label='train_loss')
        plt.legend(loc='upper right')
        plt.ylim([2.5,5])
        plt.savefig('./outputs/baseline_gender_aug.png')
        plt.show()
        
    elif baseline:
        score = gender_model.fit(train_x, train_y, epochs=epochs, batch_size=batch_size, validation_data=(test_x, test_y))
        model='_'
        pd.DataFrame.from_dict(scores).to_csv('./outputs/baseline_gender_'+model+'.csv')
        plt.plot(score.history['loss'], label='loss')
        plt.plot(score.history['val_loss'], label='val_loss')
        plt.legend(loc='upper right')   
        plt.savefig('./outputs/baseline_gender_'+model+'.png')
        plt.show()

else:
    #pre-trained weights for age prediction: https://drive.google.com/file/d/1YCox_4kJ-BYeXq27uUbasu--yz28zUMV/view?usp=sharing
    age_model.load_weights("./trained_models/gender_model_weights.h5")

## Model Prediction for Gender

In [ ]:
predictions = gender_model.predict(test_x)

In [ ]:
pred_list = []; actual_list = []

for i in predictions:
    pred_list.append(np.argmax(i))

for i in test_y: 
    actual_list.append(np.argmax(i))

confusion_matrix(actual_list, pred_list)
print(classification_report(actual_list, pred_list))